# Trajectory optimization

In this work, we will try to implement and optimize the trajectory of an expendable two-stage-to-orbit launch vehicle using simplified model.

In [ ]:
import numpy as np
import copy
import scipy
import constants as Cst
from modele_atmos import compute_atmos
import lib_trajectory
import matplotlib.pyplot as plt

# Definition of design and optimization variables

The goal is to put a payload into a LEO (250 km $\times$ 250 km) orbit. The payload mass has to be maximized.

## The control and design variables are the following
- mass of the payload (1 variable),
- delta pitch angle for the pitch over maneuver (1 variable),
- bilinear tangent law parameters for the second stage (3 variables).

## The constraints are the following
- $h(t=t_f) = h_{\text{target}}$ (tolerance of 1km)
- $v(t=t_f) = v_{\text{target}}$ (tolerance of 100 m/s)
- $\gamma(t=t_f) = \gamma_{\text{target}}$ (tolerance of 1 deg)

## Several trajectory parameters are considered as fixed for the trajectory optimization
- duration of vertical phase
- duration of pitch over maneuver phase
- duration of exponential decay (second phase of pitch over maneuver)

The trajectory is defined as follows:
* Vertical phase with the pitch angle $\theta = 90\; \text{deg}$

* A pitch-over maneuver with two phases:
   - The evolution of the pitch angle ($\theta$) to have a linear evolution of angle of attack ($\alpha = \theta - \gamma$), with respect to the following law: $\theta = \gamma - \Delta_{\theta_{po}}\frac{t}{\Delta_{t_{po}}}$  with $\gamma$ the flight path angle

   - Exponential decay of angle of attack ($\alpha$) leading to the evolution of the pitch angle: $\theta = \gamma -  \Delta_{\theta_{po}}  \text{exp}\left(-\frac{t}{\Delta_{t_{decay}}}\right)$

* A gravity turn phase in which the pitch angle is defined by $\theta = \gamma$ (i.e. $\alpha = 0$).

* A bilinear tangent law of pitch angle for exoatmospheric flight (flight of second stage) : $\theta = \text{arctan}\left(\frac{a^\xi \text{tan}(\theta_i)+(\text{tan}(\theta_f)-a^\xi \text{tan}(\theta_i)) t'}{a^\xi+(1-a^\xi)t'}\right)$ with $t'$ the normalized time within the phase, $\theta_i$ and $\theta_f$ the initial and final values of pitch angle for this phase, $a$ is fixed to 100 and $\xi$ has to be optimized.   
    
    
<img src="flight_phases.png" alt="Drawing" style="width: 500px;"/>


First, we have to define the launch vehicle model. For this exercise, we will use a Falcon 9 launcher. We consider that the fairing is jetissoned during the stage separation.

In [ ]:
Falcon_model={}
#First stage
Falcon_model['First_stage'] = {}
# Propellant mass (kg)
Falcon_model['First_stage']['Propellant_mass'] = 395700.
# Dry mass (kg)
Falcon_model['First_stage']['Dry_mass'] = 25600.
# Specific impulse in vacuum (s)
Falcon_model['First_stage']['Isp_vac'] = 312.
# Mass flow rate (kg/s)
Falcon_model['First_stage']['Mass_flow_rate'] = 9*298.8
# Reference area (m^2)
Falcon_model['First_stage']['Reference_area'] = np.pi*(3.7/2) **2
# Nozzle exit area (m^2)
Falcon_model['First_stage']['Nozzle_exit_area'] = 9*(0.92/2)**2*np.pi

#Second stage
Falcon_model['Second_stage'] = {}
# Propellant mass (kg)
Falcon_model['Second_stage']['Propellant_mass'] = 92670.
# Dry mass (kg)
Falcon_model['Second_stage']['Dry_mass'] = 3900.
# Specific impulse in vacuum (s)
Falcon_model['Second_stage']['Isp_vac'] = 348.
# Mass flow rate (kg/s)
Falcon_model['Second_stage']['Mass_flow_rate'] = 287.45
# Reference area (m^2)
Falcon_model['Second_stage']['Reference_area'] =  np.pi*(3.7/2) **2
# Nozzle exit area (m^2)
Falcon_model['Second_stage']['Nozzle_exit_area'] = (3.3/2)**2*np.pi
# Fairing mass (kg)
Falcon_model['Second_stage']['Fairing_mass'] = 1900.

In [ ]:
#Parameters for control
Parameters = {}
Parameters['Control']={}
Parameters['Control']['First_stage'] = {}
Parameters['Control']['First_stage']['Vertical_phase_duration']=8         #(s)
Parameters['Control']['First_stage']['Pitch_over_duration']=8             #(s)
Parameters['Control']['First_stage']['Pitch_over_exp_decay_duration']=10.   #(s)

In [ ]:
#Specifications of mission
Parameters['Specifications'] = {}
Parameters['Specifications']['Altitude']= 250*1e3         #(m)
Parameters['Specifications']['Flight_path_angle']= 0.     #(deg)
Parameters['Specifications']['Velocity']= 7300.           #(m/s)

Then, we have to define our optimization problem.

We will use a gradient-based optimizer called SLSQP of the `scipy` package.
[https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html)

To create the optimization problem, we need to define :
- the objective function (here the payload mass to be maximized)
- the constraint function (here the reach of the orbit)
- the bounds on the design variables

Design variables order is :
- payload mass (tons)
- initial pitch for second stage (deg)
- ksi parameter for bi linear tangent law (-)
- delta pitch parameter for first stage (deg)

We initialize the design variables to:

- $m_{CU} = 22.5\;\text{t}$
- $\theta_i = 30\;\text{deg}$
- $\xi = 0.$
- $\Delta_\theta = 2.\;\text{deg}$


In [ ]:
#Definition of objective function for trajectory

def obj_function_trajectory(design_var):
    """
    Function defining the objective function for the optimization

    :design_var: design variables of the first stage (payload mass and control law)

    """
    Payload_mass = design_var[0]*1e3
    return - Payload_mass

#Definition of constraint function for trajectory

def constraint_function_trajectory(design_var, Parameters,Rocket_model):
    """
    Function defining the constraints function for the optimization

    :design_var: design variables of the first stage (payload mass and control law)

    :Parameters: dictionary of parameters for the simulation

    :Rocket_model: dictionary composed of rocket parameters

    """

    # Integration of first stage trajectory
    flight_history_first_stage = lib_trajectory.trajectory_integration_first_stage(design_var, Parameters,Rocket_model)

    # Integration of second stage trajectory
    flight_history_second_stage = lib_trajectory.trajectory_integration_second_stage(design_var,Parameters,Rocket_model,flight_history_first_stage)


    # Calculation of discrepancy at the orbit

    discrepancy_altitude = np.abs(Parameters['Specifications']['Altitude']-flight_history_second_stage['altitude'][-1])
    discrepancy_velocity = np.abs(Parameters['Specifications']['Velocity']-flight_history_second_stage['velocity'][-1])
    discrepancy_flight_path_angle = np.abs(Parameters['Specifications']['Flight_path_angle']-flight_history_second_stage['flight_path_angle'][-1])

    #Definition of tolerances on state vector at the orbit injection
    tolerance_altitude = 1e3  #(m)
    tolerance_velocity = 100. #(m/s)
    tolerance_flight_path_angle = 1.  #(deg)

    return np.array([(tolerance_altitude-discrepancy_altitude)/1e3, tolerance_velocity-discrepancy_velocity,
                     tolerance_flight_path_angle-discrepancy_flight_path_angle])


constraint = lambda x:  constraint_function_trajectory(x,Parameters,Falcon_model)

objective = lambda x:  obj_function_trajectory(x)

design_var = np.array([22.5,30.,0.,2.])

bounds_design_var = ((20.,30.),(20.,40.),(-1,1.),(1.,3.))

Before launching the optimization process, it is interesting to simulate the trajectory for the initial valules of the control variables

In [ ]:
#Simulation of trajectory
flight_history_first_stage = lib_trajectory.trajectory_integration_first_stage(design_var, Parameters,Falcon_model)

flight_history_second_stage = lib_trajectory.trajectory_integration_second_stage(design_var,Parameters,Falcon_model,flight_history_first_stage)

Several variables are available after the simulation of the trajectory.


In [ ]:
print('available variables',flight_history_first_stage.keys())

- 'time': time of flight
- 'r': radius from the Earth center in m
- 'velocity': velocity of the launcher (m/s)
- 'flight_path_angle': fligth path angle $\gamma$ in deg
- 'longitude': longitude in deg
- 'mass': mass in kg
- 'axial_load_factor': axial load factor (-)
- 'qdyn': dynamic pressure in Pa
- 'flux': heat flux in W/m2
- 'altitude': altitude from Earth surface in m
- 'AoA': Angle of Attack in deg
- 'pitch_angle': Pitch angle in deg
- 'rho': atmosphere density (kg/m3)
- 'Pa': atmospheric pressure at a certain altitude in Pa
- 'Drag_coeff': Drag coefficient CD (-)
- 'Thrust': thrust at a certain altitude in N
- 'Thrust_vac': thrust in the vaccum in N
- 'mass_flow_rate': mass flow rate of the rocket engine in kg/s
- 'g': Gravitational Earth acceleration at a certain altitude (m/s2)


We can plot the evolution of several variables as functions of time

In [ ]:
# Plot of trajectory at initial values

#Some nice plots
plt.figure(figsize=(20,10))
plt.subplot(2,3,1)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['altitude']/1e3,flight_history_second_stage['altitude']/1e3)))
plt.title('Altitude')
plt.xlabel('Time (s)')
plt.ylabel('Altitude (km)')

plt.grid()
plt.subplot(2,3,2)

plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['velocity'],flight_history_second_stage['velocity'])))
plt.title('Velocity')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')

plt.grid()


plt.subplot(2,3,3)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['flight_path_angle'],flight_history_second_stage['flight_path_angle'])))
plt.title('Flight path angle')
plt.xlabel('Time (s)')
plt.ylabel('Flight path angle (deg)')
plt.grid()

plt.subplot(2,3,4)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['pitch_angle'],flight_history_second_stage['pitch_angle'])))
plt.title('Pitch angle')
plt.xlabel('Time (s)')
plt.ylabel('Pitch angle (deg)')

plt.grid()

plt.subplot(2,3,5)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['mass']/1e3,flight_history_second_stage['mass']/1e3)))

plt.title('Mass')
plt.xlabel('Time (s)')
plt.ylabel('Mass (t)')

plt.grid()

plt.subplot(2,3,6)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['AoA'],flight_history_second_stage['AoA'])))

plt.title('AoA')
plt.xlabel('Time (s)')
plt.ylabel('Angle of attack (deg)')

plt.grid()

We can see that the simulated trajectory does not satisfy the constraints (positive as satisfied).

In [ ]:
constraint(design_var)

Then, we launch the optimization process.

In [ ]:
# We launch the optimization
sol_opt = scipy.optimize.fmin_slsqp(objective,
                              design_var,
                              f_ieqcons = constraint,
                              bounds = bounds_design_var,
                              disp=True,
                             epsilon=1e-3)

print(sol_opt)

We can verify that the constraints are effectively satisfied:

In [ ]:
constraint(sol_opt)

In [ ]:
# We simulate the optimal solution
flight_history_first_stage = lib_trajectory.trajectory_integration_first_stage(sol_opt, Parameters,Falcon_model)
flight_history_second_stage = lib_trajectory.trajectory_integration_second_stage(sol_opt,Parameters,Falcon_model,flight_history_first_stage)

In [ ]:
# Plot of trajectory at initial values

#Simulation of trajectory
flight_history_first_stage = lib_trajectory.trajectory_integration_first_stage(sol_opt, Parameters,Falcon_model)
flight_history_second_stage = lib_trajectory.trajectory_integration_second_stage(sol_opt,Parameters,Falcon_model,flight_history_first_stage)

#Some nice plots
plt.figure(figsize=(20,10))
plt.subplot(2,3,1)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['altitude']/1e3,flight_history_second_stage['altitude']/1e3)))
plt.title('Altitude')
plt.xlabel('Time (s)')
plt.ylabel('Altitude (km)')

plt.grid()
plt.subplot(2,3,2)

plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['velocity'],flight_history_second_stage['velocity'])))
plt.title('Velocity')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')

plt.grid()


plt.subplot(2,3,3)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['flight_path_angle'],flight_history_second_stage['flight_path_angle'])))
plt.title('Flight path angle')
plt.xlabel('Time (s)')
plt.ylabel('Flight path angle (deg)')
plt.grid()

plt.subplot(2,3,4)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['pitch_angle'],flight_history_second_stage['pitch_angle'])))
plt.title('Pitch angle')
plt.xlabel('Time (s)')
plt.ylabel('Pitch angle (deg)')

plt.grid()

plt.subplot(2,3,5)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['mass']/1e3,flight_history_second_stage['mass']/1e3)))

plt.title('Mass')
plt.xlabel('Time (s)')
plt.ylabel('Mass (t)')

plt.grid()

plt.subplot(2,3,6)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['AoA'],flight_history_second_stage['AoA'])))

plt.title('AoA')
plt.xlabel('Time (s)')
plt.ylabel('Angle of attack (deg)')

plt.grid()

plt.figure()
plt.subplot(1,1,1)
plt.plot(np.concatenate((flight_history_first_stage['time'],flight_history_first_stage['time'][-1]+flight_history_second_stage['time'])),
         np.concatenate((flight_history_first_stage['axial_load_factor'],flight_history_second_stage['axial_load_factor'])))

plt.title('AoA')
plt.xlabel('Time (s)')
plt.ylabel('axial_load_factor')

plt.grid()



## Question : Compute the different losses

Losses are given :
\begin{align*}
\displaystyle \int_{t_0}^{t_f} \dot{V} dt &=& \displaystyle \int_{t_0}^{t_f} \frac{T_{vacuum}}{m} dt &-&\displaystyle \int_{t_0}^{t_f} \cos (\theta-\gamma+\delta)) \frac{Pa(r)A_n}{m} dt&-
&  \displaystyle \int_{t_0}^{t_f} (1- \cos (\theta-\gamma+\delta)) \frac{T_{vacuum}}{m} dt\\
&-&\displaystyle \int_{t_0}^{t_f} \frac{D}{m} dt &-& \displaystyle \int_{t_0}^{t_f} g(r) sin\gamma dt
\end{align*}

## Question : Plot the dynamic pressure profile

### On the dynamic pressure profile for this trajectory, we can see that he Pdyn is greater than 40kPa.

Try to reformulate the optimization problem to respect a constraint about $\text{max}(Pdyn)\leq40\text{kPa}$ and change fixed trajectory parameters to satisfy this constraint.